# A Basic Intro

This is an attempt to motivate the Kalman filter in what I hope is a more intuitive way than many presentations.  Plus, I'll add several worked examples.

# A brief outline
This workbook (will) go through several problems for a variety of reasons.
* Ready
  * Simple 1-d problem.  Show that for a system with no dynamics and trivial measurement model the equantions are actually quite simple and reduce to what we would actually guess 
  * Code for Kalman Filter. 
  * Code for Steady State Values.  
  
* To Do
  * Refer to the ambigious both optimal situation and show that it leads to a singularity
  * Simple 2-d problem.  Show how Kalman filter handles filtering in the case with states
  * Show how a straight filter can be better with no input
  * Show impact of modeling error
  * Show impact of non-gaussian error (uniform perhaps and bias)

#### Version Info
* Version 1.03
* Status content: review through Kalman filter code and simple example
* Status performance: generalized the Kalman filter code so it works symbolically or numerically
* Notes: 1.03 

## We will start with a simple 1-d problem

### Our System

We will start by estimating the acceleration of a simple 1d system.  For this reason we aren't worrying about constants or units, for this problem everything is "unity."

This has a real simple physical model
* $a_k = u_k$ 

with

* $a_k$ = acceleration at time k (the actual acceleration)
* $u_k$ = is the input at time k




We make the model  more realistic by allowing modeling error, often called process noise.  Calling it noise is an estimation and detection based way to think of it.  In that world view there are "know" or "modeled" quantities and "unknown" or "noise." If you come to this from a more engineering based approach calling it noise may seem weird, after all it is modelign error however, either option works.


* $a_k = u_k + q_k$

with

* $q_k$ = zero mean random variable representing process noise (modeling error) with covariance $Q$








### The measurement model

We will start with a trivial observation model, we add m to the subscript to indicate it is a measured quantity
* $a_{mk} = a_k + r_k$

with

* $r_k$ = zero mean random variable representing observatioin noise (measurement error) with covaraince $R$ 


Pretty much anyone would call this noise, it is simply how off your sensor is.


### Estimating Acceleration a Naive Approach

We need one little bit of additional notation.  It is pretty common in the estimation and detection literature to use $\hat *$ to indicate an estimate.  In our case 

* $\hat a_k$ is our estimation of accleration at time k.

#### The Most Basic Problem Estimate Acceleration In The No Noise No Modeling Error Case

we could now estimate accleration with either of two equations 
* $a_{mk} = a_k + r_k$
* $a_k = u_k + q_k$

In the case of no measurement error $r_k=0$ and $q_k=0$ and no modeling errors these reduce to
* $a_{mk} = a_k$
* $a_k = u_k$


It should be clear that we now have the mathematically awkward, but practically ideal situation where have two optimal estimates.  Also note they are clearly optimal under any metric, since the estimate is always EXACTLY correct
* $\hat a_k =  a_{mk}$
* $\hat a_k =  u_{k}$



#### Only measurement or modeling error

If we have only measurement error $r_k$ but still no modeling error $q_k=0$ it should be clear that we now do have a best choice, and thus a single optimal choice

* $\hat a_k = u_k$

Conversely if we have only modeling $q_k$ error and no measurement error $r_k$ we also have a single optimal choice

* $\hat a_k = a_{mk}$

Again these are still clearly optimal under any metric as they are exactly the same as the quantity we are estimating.

#### Measurement and modeling error

if systems with no measurement error and no modeling error were common engineering and experimental physics would both be trivial. Clearly, for real systems we should expect both.

* $a_{mk} = a_k + r_k$
* $a_k = u_k + q_k$

With this case we want to ask what is the optimal estimate, seems like it should be some combination of the two estimates weighted by their relative noise.

The most obvious model to propose is 
* $\hat a_k = \frac{q}{q+r} a_{mk} + \frac{r}{q+r} u_k$

With 
* no observation noise r = 0 this becomes $u_k$ 
* no modeling error q = 0 this becomes $a_{mk}$ 
* r = q this becomes $\frac{1}{2} a_{mk} + \frac{1}{2} u_k$


### Estimating Acceleration With The Kalman Filter

Now let's grind through the math of the Kalman filter and see what we get we will follow the version on Wiki
https://en.wikipedia.org/wiki/Kalman_filter

From Wiki, note this is matrix notation for now we won't worry about that and will just reduce everything to scalars 
We refer to the general form as Kalman and the equantions reduced to match our system as specalized.  We will follow the convention of vectors and matricies being capital letters.

There is one exception that we will take a momentary detour for.  If you don't like matricies you can ignore this and all references to spaces below.

#### Measurement and state space

Let't take our example but assume we have modeled both acceleration and velocity, now rather than a scalar our states are a vector

* $X_k = \left[ \begin{matrix} a_k \\ v_k \end{matrix} \right]$

We will introduce $Z$ for our measurement ($_m$) isn't a great notation for non-scalar quantities. 
Just to be difficult let's say our first sensor measures acceleration + velocity  and our second sensor measures acceleration

* $Z_k = \left[ \begin{matrix} a_k + v_k \\ a_k \end{matrix} \right]$

Expressed with the state vector

* $Z_k = \left[ \begin{matrix} 1 & 1 \\ 1 & 0 \end{matrix} \right] * X_k$

The matrix that multiplies $X$ is often called $H$ or $C$.  It should be clear that we have two state spaces here
* $X_k$ and $Z_k$ which are related by the linear transform $H$
* $Z_k = H X_k$


### Recall our system model

* $a_k = u_k + q_k$
* $a_{mk} = a_k + r_k$



### Translating notation to Kalman Filter Notation (from Wiki)

#### Dynamics

The Kalman filter equation for the dynamics is 
* $X_k = F_k X_{k-1} + B_k U_k + W_k$


* $W_k=\mathcal{N}(0,\,Q)$

Some sources will use $A$ rather than $F$ 


Referring back to our system model
* $F_k = 0 $ this means there are no dynamics the state is instantaneously equal to the input
* $B_k = 1 $ the state is equal to the input
* $W_k = q_k$ scalar normal random variable with zero mean and variance q

#### Kalman filter Wiki Notation for Measurement Model

The Kalman filter equation for the measurement model is 
* $Z_k = H_k X_k + V_k$
* $V_k=\mathcal{N}(0,\,R)$

Referring back to our system model
* $H_k = 1 $ we observe the state itself (no scaling)
* $V_k = r_k$ scalar normal random varianble with zero mean and variance r


### Kalman Filter
In this section we use "specalized" to refer to the Kalman filter applied to our exact problem

#### One more quick bit of notation
The Kalman filter has a lot of terms that are similar.  You can distiguish them by using different variable names, but this becomes a LOT of variable names to keep track of (mentally). Instead usually what is used is conditioned subscripting, which follows the noation of conditional probabilities.

For example
*$X(k)_{k|k-1}$
is read as $X_k$ given all the data to $k-1$

#### Kalman Predicted (a priori) state estimate
You can think of this as the best guess of what the state will be based on knowledge of the input $u$ and our system model $F$ and $B$. This is a priori relative to the measurment.  It is the model of the system we have.
If you are happy with spaces we note this is the state space, which in our simple problem is also the observation space:)

* Kalman $\hat X(k)_{k|k-1} = F_k \hat X(k)_{k-1|k-1} + B_Ku_k$
* Specalized $\hat x(k)_{k|k-1} = u_k$ 

#### Kalman Predicted (a priori) estimate of covariance
You can think of this as the best guess of what the new covariance will be, again just using our system model.  Again we note we are still in the state space.

* Kalman $P_{k|k−1}=F_kP_{k|k−1}F^T_k+Q_k$
* Specalized $p_{k|k−1}=q_k$

#### Kalman Innovation or measurement pre-fit residual
This is where we measure how close our prediction is to the actual measurement, where $H_k$ tells us how the states map to the measurements.  Here we are in the measurement space.

Note not all forms of the Kalman filter will define this term.  For example probabilistic robotics doesn't bother with this term.

* Kalman    $\tilde Y_k = Z_k - H_k \hat X_{k|k-1}$
* Specalized $\tilde y_k = z_k -\hat x_{k|k-1} = z_k - u_k$

#### Kalman Post Fit Covariance
This is our covariance updated based on the measurement. You can think of this as the total covariance.  Alternatively as as our modeled error (a priori) plus  error translated through the $H_k$ plus the measurement error in the measurement space $R_k$

We are in the measurement space, this is clear by the fact that we are adding $R_k$
* Kalman $S_k=H_kP_{k|k−1}H^T_k+R_k$
* Specalized $S_k=P_{k|k−1}+R_k =q_k+r_k$



#### Kalman Optimal Kalman Gain
This can then be thought of as two terms $P_k$ our modeling error and $S_k$ our total error including measurement error with $H^T$ mapping this back to the state space

Now we want to end up back in State space here so that is why we have $H^T_k$ before $S$ to translate S back to state space
* Kalman $K_k = P_{k|k-1} H^T_k S^{-1}_k$
* Noting that $H$ = 1 as derived in the system equivalent
* Specalized $K_k = P_{k|k-1}  S^{-1}_k = q_k  (q_k + r_k)^{-1}$


#### Kalman Optimal Estimate of State
* Updated (a posteriori) state estimate
* Kalman $x_{k|k} = \hat x_{k|k} + K_k* \tilde y_k$
* Specalized $x_{k|k} = u_k + \frac{q_k}{q_k + r_k} * (z_k - u_k)$



So an obvious quesiton is is this the same as our intuitive model?
* Kalman Specalized $x_{k|k} = u_k + \frac{Q_k}{Q_k + R_k} * (z_k - u_k)$
* Intuitive $\hat a_k = \frac{q}{q+r} a_{mk} + \frac{r}{q+r} u_k$

If we simply add and subtract $\frac{q}{q+r}u$ we get
* $\hat a_k = \frac{q}{q+r} a_{mk} + \frac{r}{q+r} u_k + \frac{q}{q+r}u - \frac{q}{q+r}u$
* $=\frac{q}{q+r} a_{mk} - \frac{q}{q+r}u + \frac{r}{q+r} u_k + \frac{q}{q+r}u $
* $=\frac{q}{q+r} (a_{mk} - u) + u_k  $

So as one might expect they are equal.

This may seem like an unsatisfying answer, since we had a ton of mathematical structure in the Kalman filter and it all came down to a simple solution.  This however is the wrong way to think about it.  Instead, it's great that our intuition matches the Kalman filter and we have an idea about what each term does.  

Further, if we look at simplicication $H$ being identity is not too bad.   While we often don't get to measure all the states it isn't a crazy assumption. It is just how complexly the measurement space is related to the state space.  

However, we should note that $F$ being zero is a pretty sever assumption and a non-zero $F$ is what we will look at next.  

It is also interesting to think about different optimality criterion.  Let's assume that bosth the measurement and process noise are high, but the known input $u$ is always zero.  The Kalman filter will still be the optimal instantenious estimate of the state however, if we want the best steady state value it's clear that we rather have an additional filter on the values.

#### What's up with all the "optimal" stuff

The Kalman Filter is the optimal linear estimator of the mean squared error of the states given known covariances.  It is useful to know it is optimal because for the stated conditions you know you don't need to look for another better algorithm.  It is also useful to notice which of these are "stronng" assumptions.  

* The most restrictive assumption is that the initial system is linear.  The extended Kalman filter reduces that restriction at the cost not just of optimality but even covergence, except under certain limiting cases.  This makes sense, you extend the class of problems handled but not to everything.

* The next strongest is reducing the problem to linear estimators.  We can think of all algorithms as being part of the class of non-linear estimators.  Withing this class there is a small class of linear estimators.  The primary limitation of linear estimators is the inability to discard completely bogus data.  To do so you need some type of nonlinearity.

* Finally we bring in that it is an estimator.  This means that you can't have future knowledge impact previous states.  You can argue that this is one of the primary reasons that graph based algorithms for SLAM surplanted Kalman filter algorithms, the ability to update past values with future data.



## A Functional Kalman Filter
The above discussion should be motivational, but the Kalman filter is itterative so it would be far better to have it written as a function rather than cells.

We will actually define it so it can be used symbolically or numerically.  We will make use of the symbolic functionality to make sure we have coded it to match what we have above.

I don't think it is particularly important to follow through the code, though if you are looking for examples of how to do this in Python it could be handy.

In [21]:
#Regeneralizing for symbolic
import numpy as np
import sympy as sym
import random
from scipy import linalg

#Utility fcn
def print_matrix(lcl_header_str, lcl_matrix):
    if (isinstance(lcl_matrix, sym.Matrix)):
        lcl_outmtrx = str(lcl_matrix)
    else:
        lcl_matrix =  np.matrix(lcl_matrix).astype(np.float64)
    lcl_blanks = " "*len(lcl_header_str)
    lcl_outmtrx = lcl_outmtrx.replace('\n','\n'+lcl_blanks)
    #outmtrx.replace('\n','\n'+blanks)
    print(lcl_header_str, lcl_outmtrx)

#Also repeat some definitions to make things more free standing
gDef_Noise = 0.01

#Since workbooks invite global variables we will use pedantic in function naming convention of lcl_ for local variables to avoid any
#accidental collision with globals

class Dynamic_System:
    #For now no looping in this level
    def __init__(self, lcl_X_init, lcl_A, lcl_B, lcl_R, lcl_C, lcl_Q, lcl_verbose=0):
        #Copy over verbose
        self.verbose = lcl_verbose
        #Initial state
        self.X = lcl_X_init
        if self.verbose>3:
            print("Initial X", self.X)
        #State transition
        self.A = lcl_A
        #Inputs
        self.B = lcl_B
        #Noise on dynamics
        self.R = lcl_R
        #Observation model
        self.C = lcl_C
        #Noise on observation
        self.Q = lcl_Q
        #Make random numbers repeatible
        self.flatten = sym.Matrix.ones(self.Q.shape[0],1)
        random.seed(0)
    def calc_cov(self, lcl_Cov):
        #In terms of std not variance
        #return (random.gauss(0*lcl_Cov,lcl_Cov*lcl_Cov)*self.flatten)
        lcl_Std = lcl_Cov**(1/2)
        #Somewhat surprisingly trying this a duck typing doesn't work the random.gauss just get's magically appended
        if (isinstance(lcl_Std, sym.Matrix)):
            return (lcl_Std)
        else:
            return (random.gauss(0*lcl_Std,lcl_Std)*self.flatten)
        
    def get_model(self):
        return(self.A, self.B, self.R, self.C, self.Q)
    def get_dynamics(self):
        return(self.A, self.B, self.C)
    def dynamics(self, lcl_input, lcl_X):
        lcl_dyn_upt = self.A * lcl_X + self.B * lcl_input
        if self.verbose>=2:
            print("Dynamics Update")
            print(lcl_dyn_upt)
        return(lcl_dyn_upt)
    def update(self, lcl_input):
        #Evolve state
        #for current_input in input:
        #    print ("Current Input ")
        #    print (current_input)
        #    print(current_input.shape)

        lcl_dyn_err = self.calc_cov(self.R)
            
        if self.verbose>3:
            print(" DYNAMICS_UPDATE")
            print("Model Error ", self.R)
            print("Dynamics Error", lcl_dyn_err)
            print("input", lcl_input)
            print("state before update", self.X)

        self.X = self.dynamics(lcl_input, self.X + lcl_dyn_err)

        if self.verbose>3:
            print("state after update", self.X)

            
            
    def show(self):
        print ("Initial state")
        print (self.X)
        print ("A -- dynamics")
        print (self.A)
        print ("B -- impact of input")
        print (self.B)
        print ("R -- assumed dynamic errors")
        print (self.R)
        print ("C -- state observation model")
        print (self.C)
        print ("Q -- noise on state observation")
        print (self.Q)
    def state(self):
        #Return state, useful primarly for debugging
        return(self.X)
    def observe(self):
        #Return observation, should generally be used rather than state
        #return(self.C*self.X+self.calc_cov(self.Q)*self.flatten)
        if self.verbose>3:
            print("C", self.C)
            print("X", self.X)
            print("Q", self.Q)
            print("Cov")
            print(self.calc_cov(self.Q))
        return(self.C*self.X+self.calc_cov(self.Q))

class Kalman_Filter:
    #Loops are done here
    def __init__(self, lcl_Dynamic_System, lcl_Sigma_Init, lcl_X_init, lcl_R, lcl_Q, lcl_verbose=0):
        self.verbose = lcl_verbose
        self.Dynamic_System = lcl_Dynamic_System
        self.Sigma = lcl_Sigma_Init
        self.X = lcl_X_init
        self.R = lcl_R
        self.Q = lcl_Q
        if self.verbose>3:
            print("Dynamic System")
            self.Dynamic_System.show()
            print("Kalman System")
            print ("  Initial State Kalman Filter ", self.X)
            print ("  R", self.R)
            print ("  Q", self.Q)

        
    def update(self, lcl_input):

        #Extract required info from dynamic system
        lcl_A, lcl_B, lcl_C= self.Dynamic_System.get_dynamics()
        
        #Create empty vectors
        self.X_act_hist = [self.Dynamic_System.state()]
        self.X_filt_hist = [self.X]
        self.K_hist = [self.Sigma]

        for lcl_current_input in lcl_input:
            if self.verbose>2:
                print("")
                print("KALMAN FILTER CALCULATION")
                print("")
                print("  Current Input", lcl_current_input)
            #Evolve the actual state
            self.Dynamic_System.update(lcl_current_input)

            #Calculate Kalman filter following https://en.wikipedia.org/wiki/Kalman_filter

            
            ##Predict
            ###Compute Predicted X  (predicted a priori state estimte)
            lcl_X_pred = self.Dynamic_System.dynamics(lcl_current_input, self.X)
            if self.verbose>2:
                print("")
                print ("  Predicted X", lcl_X_pred)
            ####First step in covariance calculation based on dynamics and dynamics "noise"
            if self.verbose>3:
                print ("Sigma")
                print (self.Sigma)

            ###Compute Predicted P covariance
            lcl_Sigma_Step_1 = lcl_A*self.Sigma*lcl_A.T+self.Q
            if self.verbose>2:
                print ("  Predicted Covariance", lcl_Sigma_Step_1)
            

            #Observe the system
            lcl_Z = self.Dynamic_System.observe()
            if self.verbose>3:
                print ("    Observed Value")
                print (lcl_Z)
            
            
            ##Update

            ###Calculate innovation pre-fit residual (error)
            lcl_Y = lcl_Z - lcl_C * lcl_X_pred

            ###Calculate innovation (or pre-fit redidual) covariance
            #self.Sigma = (sym.Matrix.eye(2)-lcl_K*lcl_C)*lcl_Sigma_Step_1
            self.Sigma = lcl_C * lcl_Sigma_Step_1 * lcl_C.T + self.R
            if self.verbose>2:
                print("  Innovation Covariance ", self.Sigma)
            
            #Compute the Kalman gain (used to weight observations and measurements)
            #lcl_K = lcl_Sigma_Step_1 * lcl_C.T * (lcl_C*lcl_Sigma_Step_1*lcl_C.T + self.Q).inv()
            lcl_K = lcl_Sigma_Step_1 * lcl_C.T * self.Sigma.inv()
            if self.verbose>2:
                print ("  Updated Kalman Gain", lcl_K)
                
            ###Updated (a posteriori) state estimate
            lcl_X_filt = lcl_X_pred + lcl_K * (lcl_Y)
            self.X = lcl_X_filt
            
            ###Updated (a posterriori) estimate of covariance
            ###Get size of matrix
            lcl_temp = lcl_K*lcl_C
            ###Possible error, have to check if this must be square
            lcl_size = lcl_temp.shape[1]
            self.Sigma = (sym.Matrix.eye(lcl_size)-lcl_K*lcl_C)*lcl_Sigma_Step_1
            if self.verbose>2:
                print ("  Updated (a posteriori) estimate of covariance", self.Sigma)
            
            #Store relevant information in a vector
            self.X_act_hist.append(self.Dynamic_System.state())
            self.X_filt_hist.append(lcl_X_filt)
            self.K_hist.append(lcl_K)
            
        if self.verbose>=1:
            print("Final Values")
            print_matrix(" X                          ", self.Dynamic_System.state())
            print_matrix(" Kalman Estimate of State X ", lcl_X_filt)
            print_matrix(" Final Covariance S(igma)   ", self.Sigma)
            print_matrix(" Final Kalman Gains K       ", lcl_K)
    #def show(self):
        #for idx in range(length(self.X_act_hist)):
        #    print("State")
        #    print(self.X_act_hist.append[idx])
        #    print("Kalman Estimate")
        #    print(self.X_filt_hist[idx])
        #    print("Kalman Gain")
        #    print(self.K_hist[idx])

def asymptotic_kal(lcl_A=[], lcl_B=[], lcl_R=[], lcl_C=[], lcl_Q=[], lcl_verbose=0):
    lcl_An = np.matrix(lcl_A).astype(np.float64)
    lcl_Bn = np.matrix(lcl_B).astype(np.float64)
    lcl_Cn = np.matrix(lcl_C).astype(np.float64)
    lcl_Qn = np.matrix(lcl_Q).astype(np.float64)
    lcl_Rn = np.matrix(lcl_R).astype(np.float64)
    
    lcl_gP = linalg.solve_discrete_are(lcl_An, lcl_Cn, lcl_Qn, lcl_Rn)
    #Change of notation
    #gFn1 = gA1
    #gGn1 = gB1
    #gHn1 = gC1
    #gRn1 = gR11
    lcl_K = lcl_gP*lcl_Cn.T*((lcl_Cn*lcl_gP*lcl_Cn.T + lcl_Rn).I)
    lcl_S = lcl_gP-lcl_K*lcl_Cn*lcl_gP
    lcl_P = lcl_An*lcl_S*lcl_An.T + lcl_Bn*lcl_Qn*lcl_Bn.T
    if lcl_verbose>0:
        print("Asymptotic Evaluation of: ")
        print_matrix(" Covariance Matrix S(igma) ", lcl_S)
        print_matrix(" Kalman Gain K             ", lcl_K)
    return(lcl_S, lcl_K, lcl_P)
        
#Define a utility function to setup up consistent dynamics and Kalman Filter
def matched_system(lcl_A=[], lcl_B=[], lcl_X=[], lcl_R=[], lcl_C=[], lcl_Q=[],  lcl_Sigma=[], lcl_verbose=0):
    #We note that the noise for our dynamic system is defined in terms for std not var
    lcl_R2 = np.matrix(lcl_R).astype(np.float64)
    #look into
    #lcl_R2 = linalg.sqrtm(lcl_R2)
    lcl_Q2 = np.matrix(lcl_Q).astype(np.float64)
    #lcl_Q2 = linalg.sqrtm(lcl_Q2)
    
    lcl_dyn_sys = Dynamic_System(lcl_X, lcl_A, lcl_B, lcl_R2, lcl_C, lcl_Q2, lcl_verbose=lcl_verbose)
    lcl_kal = Kalman_Filter(lcl_dyn_sys, lcl_Sigma, lcl_X, lcl_R, lcl_Q, lcl_verbose=lcl_verbose)
    asymptotic_kal(lcl_A, lcl_B, lcl_R, lcl_C, lcl_Q,  lcl_verbose)
    return(lcl_kal)


def repeat_input(input, times):
    input_list = [input]
    for idx in range(times-1):
        input_list.append(input)
    return(input_list)



### Let's use this on our simple system

We set verbose to 0 for the dynamic system, we don't want to see any of it's calculations
We set verbose to 3 for the kalman filter, we do want to see it's intermediate calculations

Also since this is done for us we relax the assumption on zero initial state and covariance matrix we used above

In [22]:
#Initial state
gx_init =  sym.symbols('xi')
gX_init  = sym.Matrix([[gx_init]])
#Dynamics
gA = sym.Matrix([[1]])
#Inputs
gB = sym.Matrix([[1]])
#Mapping of states to measurements
gC = sym.Matrix([[1]])
#Noise on dynamics
gr =  sym.symbols('r')
gR = sym.Matrix([[gr**2]])
#Noise on measurements
gq =  sym.symbols('q')
gQ = sym.Matrix([[gq**2]])
gDyn_sys = Dynamic_System(gX_init, gA, gB, gR, gC, gQ, lcl_verbose=0)
#Create a single input
gu = sym.symbols('u')
gsingle_input = repeat_input(sym.Matrix([[gu]]),1)
#Initial covariance matrix
gsigma_init = sym.symbols('si')
gSigma_Init =  sym.Matrix([[gsigma_init]])
#Call the raw calman filter
gKal = Kalman_Filter(gDyn_sys, gSigma_Init, gX_init, gR, gQ, lcl_verbose=3)

#gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR, lcl_C=gC, lcl_Q=gQ, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(gsingle_input)



KALMAN FILTER CALCULATION

  Current Input Matrix([[u]])

  Predicted X Matrix([[u + xi]])
  Predicted Covariance Matrix([[q**2 + si]])
  Innovation Covariance  Matrix([[q**2 + r**2 + si]])
  Updated Kalman Gain Matrix([[(q**2 + si)/(q**2 + r**2 + si)]])
  Updated (a posteriori) estimate of covariance Matrix([[(q**2 + si)*(-(q**2 + si)/(q**2 + r**2 + si) + 1)]])
Final Values
 X                           Matrix([[u + xi + (r**2)**0.5]])
 Kalman Estimate of State X  Matrix([[u + xi + (q**2 + si)*((q**2)**0.5 + (r**2)**0.5)/(q**2 + r**2 + si)]])
 Final Covariance S(igma)    Matrix([[(q**2 + si)*(-(q**2 + si)/(q**2 + r**2 + si) + 1)]])
 Final Kalman Gains K        Matrix([[(q**2 + si)/(q**2 + r**2 + si)]])


So as expected the calcualted Kalman gain matches what we derived piece by piece.


# End Of Vetted Part of Sheet
Note actually the Kalman filter code below works, but I haven't really thought of how I want to tie it all together.

# My next steps are
1. Put in a simulation for the case mentioned above
2. Work through an example with dynamics
Note the rest of the sheet is somewhat unstructured.  Thought the code for a Kalman filter works fine.

In [ ]:
* Specalized $x_{k|k} = u_k + \frac{Q_k}{Q_k + R_k} * (z_k - u_k)$


* If we take R=0 then this reduces to 
* Specalized $x_{k|k} = u_k + 1 * (z_k - u_k) = z_k$
* If we take R>0 and Q=0
* Specalized $x_{k|k} = u_k + 1 * (z_k - u_k) = u_k$




Next step put in some modeling dynamics


### A Basic Model

Some general notes
--The naming conventions in this worksheet are NOT recommended for general work.  Worksheets, particularly worksheets to be shared have the high risk of accidentally using a global variable when a local variable was intended.  This is particularly true because the generic namespace is the global name space.  I would NOT recommend following this convention in any conventional programming.
--Obviously this is python and it's libraries, if you aren't familiar with them don't worry you can just ignore the programming steps and follow along with the description
--The markdown uses Latex, if you are familiar great, if not I wouldn't worry too much about learning it.  Latex has a sort of steep entry curve

We start with the most basic system
--Robot on a plane
--Linear dynamics
--No noise
--Trivial coordinates


Our dynamics are 
Standard dynamics are:

$ x_{k} = A_k x_{k-1} + B_ku_k + w_k$


Note some other letters are $A, B = F, G$ and $w = r$



We then observe the state with

$ z_k = C_kx_k + v_k $


Note some alternative letters for $H = C$ and $q = v$.  The "hat" notation is pretty standard for estimate.


$\hat x(k)_{k|k-1} = F_k \hat x(k)_{k-1|k-1} + B_Ku_k$

or

$\hat x(k)_{k|k-1} = A_k \hat x(k)_{k-1|k-1} + B_Ku_k$





Now we run into a classic problem we want to do symbolic math not numeric.  Maple or Mathematic would be a better place to do this but I think we will be able to get by in Python, let's find out.
https://paulpotgieter.org/2020/10/30/solving-symbolic-matrix-equations-in-python-with-sympy/
http://scipy-lectures.org/packages/sympy.html

In [1]:
import numpy as np
import sympy as sym
import random
from scipy import linalg

## Define model without noise

We are defining the simpliest discrete time model.  There are two independent states, two independent controls, and two independent observations.  In discrete time this drives a model that is mostly just identity matricies

$Ax$

Define the dynamics, how state evolves based on past state (the homogenious part of a differential equation)

$A = \begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}$

In [2]:
gA = sym.Matrix([[1, 0], [0, 1]])
gx, gy = sym.symbols('x, y')
gX = sym.Matrix([[gx], [gy]])
gA*gX

Matrix([
[x],
[y]])

$Bu$

$B = \begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}$

Define how the state evolves based on the input

In [3]:
gB = sym.Matrix([[1, 0], [0, 1]])
gux, guy = sym.symbols('ux, uy')
gU = sym.Matrix([[gux], [guy]])
gB*gU

Matrix([
[ux],
[uy]])

$z_k = C_k x_k $

$C = \begin{matrix} 1 & 0 \\ 0 & 1 \end{matrix}$

Define the observation model

In [4]:
gC = sym.Matrix([[1, 0], [0, 1]])
gC*(gA*gX+gB*gU)

Matrix([
[ux + x],
[uy + y]])

Test this out, note this appears to print in alphabetic order which is kind of a bummer but oh well
What this says is the the observation is exactly the state, not surprising in this trivial example

## Adding Noise

Define the R matrix, this is our assumed noise model for the dynamics

In [5]:
grx, gry = sym.symbols('rx, ry')
gR = sym.Matrix([[grx, 0], [0, gry]])
gR

Matrix([
[rx,  0],
[ 0, ry]])

Define the Q matrix, this is our assumed noise model for the measurements

In [14]:
gqx, gqy = sym.symbols('qx, qy')
gQ = sym.Matrix([[gqx, 0], [0, gqy]])
gQ

Matrix([
[qx,  0],
[ 0, qy]])

Define the specific noise models
--start with noise free


In [10]:
gQn = gQ.subs('qx', 0).subs('qy', 0)
gRn = gR.subs('rx', 0).subs('ry', 0)
gQn

Matrix([
[0, 0],
[0, 0]])

## DEFINE THE KALMAN FILTER

Define the Sigma Matrix, this keeps track of the variance

In [10]:
gsx, gsy = sym.symbols('sx, sy')
gSigma = sym.Matrix([[gsx, 0], [0, gsy]])
gSigma

Matrix([
[sx,  0],
[ 0, sy]])

Initial State Estimate

$x = \begin{matrix} 0 \\ 0 \end{matrix}$

In [11]:
gX = sym.Matrix([[0], [0]])

Kalman Loop (note .T is transpose)

### Compute Predicted X  (predicted a priori state estimte)


$\hat x(k)_{k|k-1} = F_k \hat x(k)_{k-1|k-1} + B_Ku_k$

$\hat x(k)_{k|k-1} = A_k \hat x(k)_{k-1|k-1} + B_Ku_k$

Note that the exact discrete time indicies may vary depending on formulation but it should take the form of current x is A* past x + B * current input

As expected since our initial state was 0, 0 this is just the inputs

In [16]:
   gX_pred = gA*gX + gB*gU
   gX_pred

Matrix([
[ux],
[uy]])

### Compute Predicted P covariance

$P_{k|k-1} = F_k P_{k|k-1} F^T_k +Q_k$

or

$\Sigma_{k|k-1} = A_k \Sigma_{k|k-1} A^T_k +Q_k$



In [17]:
  gSigma_Step_1s = gA*gSigma*gA.T+gR
  gSigma_Step_1s
  

Matrix([
[rx + sx,       0],
[      0, ry + sy]])

### Innovation or pre-fit residual

$S_k = H_k P_{k|k-1} H^T_k + R_k$

or 

$\Sigma_{k} = C_k \Sigma_{k|k-1} C^T_k + R_k$

In [18]:
  
gSigma_Step_1n = gA*gSigma*gA.T+gRn
gSigma_Step_1n

Matrix([
[sx,  0],
[ 0, sy]])

### Compute The Kalman Gain

$K_k = P_{k|k-1} H^T_k P^{-1}_k$

$K_k = \Sigma_{k|k-1} C^T_k \Sigma^{-1}_k$


In [20]:
  gKs = gSigma_Step_1s * gC.T * (gC*gSigma_Step_1s*gC.T + gQ).inv()
  gKs

Matrix([
[(rx + sx)/(qx + rx + sx),                        0],
[                       0, (ry + sy)/(qy + ry + sy)]])

--Compute Kalman Gain, weighting between model and measurements for our specific case


In [22]:
  gKn = gSigma_Step_1n * gC.T * (gC*gSigma_Step_1n*gC.T + gQn).inv()
  gKn

Matrix([
[1, 0],
[0, 1]])

  Actually in this case any Kalman gain is as good as any other one

  Compute X based on the measurement, Kalman gain, and predection

  Create an observation, based on our noise model

In [23]:
  gZ = gC*gX+gQn*sym.Matrix([[1], [1]])
  gZ

Matrix([
[0],
[0]])

  Update the filtered value of X

In [24]:
  gX_filt = gX_pred + gKn * (gZ - gC*gX_pred)
  gX_filt

Matrix([
[0],
[0]])

  Update the value of the Kalman Filter

In [25]:
  gSigma_Finaln = (sym.Matrix.eye(2)-gKn*gC)*gSigma_Step_1n
  gSigma_Finaln


Matrix([
[0, 0],
[0, 0]])

Note we now have a zero matrix

Let's make this more computable we will define a class

In [26]:
gR

Matrix([
[rx,  0],
[ 0, ry]])

Define some functions so we can do this itteratively

In [21]:
#Regeneralizing for symbolic
import numpy as np
import sympy as sym
import random
from scipy import linalg

#Utility fcn
def print_matrix(lcl_header_str, lcl_matrix):
    if (isinstance(lcl_matrix, sym.Matrix)):
        lcl_outmtrx = str(lcl_matrix)
    else:
        lcl_matrix =  np.matrix(lcl_matrix).astype(np.float64)
    lcl_blanks = " "*len(lcl_header_str)
    lcl_outmtrx = lcl_outmtrx.replace('\n','\n'+lcl_blanks)
    #outmtrx.replace('\n','\n'+blanks)
    print(lcl_header_str, lcl_outmtrx)

#Also repeat some definitions to make things more free standing
gDef_Noise = 0.01

#Since workbooks invite global variables we will use pedantic in function naming convention of lcl_ for local variables to avoid any
#accidental collision with globals

class Dynamic_System:
    #For now no looping in this level
    def __init__(self, lcl_X_init, lcl_A, lcl_B, lcl_R, lcl_C, lcl_Q, lcl_verbose=0):
        #Copy over verbose
        self.verbose = lcl_verbose
        #Initial state
        self.X = lcl_X_init
        if self.verbose>3:
            print("Initial X", self.X)
        #State transition
        self.A = lcl_A
        #Inputs
        self.B = lcl_B
        #Noise on dynamics
        self.R = lcl_R
        #Observation model
        self.C = lcl_C
        #Noise on observation
        self.Q = lcl_Q
        #Make random numbers repeatible
        self.flatten = sym.Matrix.ones(self.Q.shape[0],1)
        random.seed(0)
    def calc_cov(self, lcl_Cov):
        #In terms of std not variance
        #return (random.gauss(0*lcl_Cov,lcl_Cov*lcl_Cov)*self.flatten)
        lcl_Std = lcl_Cov**(1/2)
        #Somewhat surprisingly trying this a duck typing doesn't work the random.gauss just get's magically appended
        if (isinstance(lcl_Std, sym.Matrix)):
            return (lcl_Std)
        else:
            return (random.gauss(0*lcl_Std,lcl_Std)*self.flatten)
        
    def get_model(self):
        return(self.A, self.B, self.R, self.C, self.Q)
    def get_dynamics(self):
        return(self.A, self.B, self.C)
    def dynamics(self, lcl_input, lcl_X):
        lcl_dyn_upt = self.A * lcl_X + self.B * lcl_input
        if self.verbose>=2:
            print("Dynamics Update")
            print(lcl_dyn_upt)
        return(lcl_dyn_upt)
    def update(self, lcl_input):
        #Evolve state
        #for current_input in input:
        #    print ("Current Input ")
        #    print (current_input)
        #    print(current_input.shape)

        lcl_dyn_err = self.calc_cov(self.R)
            
        if self.verbose>3:
            print(" DYNAMICS_UPDATE")
            print("Model Error ", self.R)
            print("Dynamics Error", lcl_dyn_err)
            print("input", lcl_input)
            print("state before update", self.X)

        self.X = self.dynamics(lcl_input, self.X + lcl_dyn_err)

        if self.verbose>3:
            print("state after update", self.X)

            
            
    def show(self):
        print ("Initial state")
        print (self.X)
        print ("A -- dynamics")
        print (self.A)
        print ("B -- impact of input")
        print (self.B)
        print ("R -- assumed dynamic errors")
        print (self.R)
        print ("C -- state observation model")
        print (self.C)
        print ("Q -- noise on state observation")
        print (self.Q)
    def state(self):
        #Return state, useful primarly for debugging
        return(self.X)
    def observe(self):
        #Return observation, should generally be used rather than state
        #return(self.C*self.X+self.calc_cov(self.Q)*self.flatten)
        if self.verbose>3:
            print("C", self.C)
            print("X", self.X)
            print("Q", self.Q)
            print("Cov")
            print(self.calc_cov(self.Q))
        return(self.C*self.X+self.calc_cov(self.Q))

class Kalman_Filter:
    #Loops are done here
    def __init__(self, lcl_Dynamic_System, lcl_Sigma_Init, lcl_X_init, lcl_R, lcl_Q, lcl_verbose=0):
        self.verbose = lcl_verbose
        self.Dynamic_System = lcl_Dynamic_System
        self.Sigma = lcl_Sigma_Init
        self.X = lcl_X_init
        self.R = lcl_R
        self.Q = lcl_Q
        if self.verbose>3:
            print("Dynamic System")
            self.Dynamic_System.show()
            print("Kalman System")
            print ("  Initial State Kalman Filter ", self.X)
            print ("  R", self.R)
            print ("  Q", self.Q)

        
    def update(self, lcl_input):

        #Extract required info from dynamic system
        lcl_A, lcl_B, lcl_C= self.Dynamic_System.get_dynamics()
        
        #Create empty vectors
        self.X_act_hist = [self.Dynamic_System.state()]
        self.X_filt_hist = [self.X]
        self.K_hist = [self.Sigma]

        for lcl_current_input in lcl_input:
            if self.verbose>2:
                print("")
                print("KALMAN FILTER CALCULATION")
                print("")
                print("  Current Input", lcl_current_input)
            #Evolve the actual state
            self.Dynamic_System.update(lcl_current_input)

            #Calculate Kalman filter following https://en.wikipedia.org/wiki/Kalman_filter

            
            ##Predict
            ###Compute Predicted X  (predicted a priori state estimte)
            lcl_X_pred = self.Dynamic_System.dynamics(lcl_current_input, self.X)
            if self.verbose>2:
                print("")
                print ("  Predicted X", lcl_X_pred)
            ####First step in covariance calculation based on dynamics and dynamics "noise"
            if self.verbose>3:
                print ("Sigma")
                print (self.Sigma)

            ###Compute Predicted P covariance
            lcl_Sigma_Step_1 = lcl_A*self.Sigma*lcl_A.T+self.Q
            if self.verbose>2:
                print ("  Predicted Covariance", lcl_Sigma_Step_1)
            

            #Observe the system
            lcl_Z = self.Dynamic_System.observe()
            if self.verbose>3:
                print ("    Observed Value")
                print (lcl_Z)
            
            
            ##Update

            ###Calculate innovation pre-fit residual (error)
            lcl_Y = lcl_Z - lcl_C * lcl_X_pred

            ###Calculate innovation (or pre-fit redidual) covariance
            #self.Sigma = (sym.Matrix.eye(2)-lcl_K*lcl_C)*lcl_Sigma_Step_1
            self.Sigma = lcl_C * lcl_Sigma_Step_1 * lcl_C.T + self.R
            if self.verbose>2:
                print("  Innovation Covariance ", self.Sigma)
            
            #Compute the Kalman gain (used to weight observations and measurements)
            #lcl_K = lcl_Sigma_Step_1 * lcl_C.T * (lcl_C*lcl_Sigma_Step_1*lcl_C.T + self.Q).inv()
            lcl_K = lcl_Sigma_Step_1 * lcl_C.T * self.Sigma.inv()
            if self.verbose>2:
                print ("  Updated Kalman Gain", lcl_K)
                
            ###Updated (a posteriori) state estimate
            lcl_X_filt = lcl_X_pred + lcl_K * (lcl_Y)
            self.X = lcl_X_filt
            
            ###Updated (a posterriori) estimate of covariance
            ###Get size of matrix
            lcl_temp = lcl_K*lcl_C
            ###Possible error, have to check if this must be square
            lcl_size = lcl_temp.shape[1]
            self.Sigma = (sym.Matrix.eye(lcl_size)-lcl_K*lcl_C)*lcl_Sigma_Step_1
            if self.verbose>2:
                print ("  Updated (a posteriori) estimate of covariance", self.Sigma)
            
            #Store relevant information in a vector
            self.X_act_hist.append(self.Dynamic_System.state())
            self.X_filt_hist.append(lcl_X_filt)
            self.K_hist.append(lcl_K)
            
        if self.verbose>=1:
            print("Final Values")
            print_matrix(" X                          ", self.Dynamic_System.state())
            print_matrix(" Kalman Estimate of State X ", lcl_X_filt)
            print_matrix(" Final Covariance S(igma)   ", self.Sigma)
            print_matrix(" Final Kalman Gains K       ", lcl_K)
    #def show(self):
        #for idx in range(length(self.X_act_hist)):
        #    print("State")
        #    print(self.X_act_hist.append[idx])
        #    print("Kalman Estimate")
        #    print(self.X_filt_hist[idx])
        #    print("Kalman Gain")
        #    print(self.K_hist[idx])

def asymptotic_kal(lcl_A=[], lcl_B=[], lcl_R=[], lcl_C=[], lcl_Q=[], lcl_verbose=0):
    lcl_An = np.matrix(lcl_A).astype(np.float64)
    lcl_Bn = np.matrix(lcl_B).astype(np.float64)
    lcl_Cn = np.matrix(lcl_C).astype(np.float64)
    lcl_Qn = np.matrix(lcl_Q).astype(np.float64)
    lcl_Rn = np.matrix(lcl_R).astype(np.float64)
    
    lcl_gP = linalg.solve_discrete_are(lcl_An, lcl_Cn, lcl_Qn, lcl_Rn)
    #Change of notation
    #gFn1 = gA1
    #gGn1 = gB1
    #gHn1 = gC1
    #gRn1 = gR11
    lcl_K = lcl_gP*lcl_Cn.T*((lcl_Cn*lcl_gP*lcl_Cn.T + lcl_Rn).I)
    lcl_S = lcl_gP-lcl_K*lcl_Cn*lcl_gP
    lcl_P = lcl_An*lcl_S*lcl_An.T + lcl_Bn*lcl_Qn*lcl_Bn.T
    if lcl_verbose>0:
        print("Asymptotic Evaluation of: ")
        print_matrix(" Covariance Matrix S(igma) ", lcl_S)
        print_matrix(" Kalman Gain K             ", lcl_K)
    return(lcl_S, lcl_K, lcl_P)
        
#Define a utility function to setup up consistent dynamics and Kalman Filter
def matched_system(lcl_A=[], lcl_B=[], lcl_X=[], lcl_R=[], lcl_C=[], lcl_Q=[],  lcl_Sigma=[], lcl_verbose=0):
    #We note that the noise for our dynamic system is defined in terms for std not var
    lcl_R2 = np.matrix(lcl_R).astype(np.float64)
    #look into
    #lcl_R2 = linalg.sqrtm(lcl_R2)
    lcl_Q2 = np.matrix(lcl_Q).astype(np.float64)
    #lcl_Q2 = linalg.sqrtm(lcl_Q2)
    
    lcl_dyn_sys = Dynamic_System(lcl_X, lcl_A, lcl_B, lcl_R2, lcl_C, lcl_Q2, lcl_verbose=lcl_verbose)
    lcl_kal = Kalman_Filter(lcl_dyn_sys, lcl_Sigma, lcl_X, lcl_R, lcl_Q, lcl_verbose=lcl_verbose)
    asymptotic_kal(lcl_A, lcl_B, lcl_R, lcl_C, lcl_Q,  lcl_verbose)
    return(lcl_kal)


def repeat_input(input, times):
    input_list = [input]
    for idx in range(times-1):
        input_list.append(input)
    return(input_list)



### Let's use this on our simple system

In [22]:
#Initial state
g_verbose = 3
gx_init =  sym.symbols('xi')
gX_init  = sym.Matrix([[gx_init]])
#Dynamics
gA = sym.Matrix([[1]])
#Inputs
gB = sym.Matrix([[1]])
#Mapping of states to measurements
gC = sym.Matrix([[1]])
#Noise on dynamics
gr =  sym.symbols('r')
gR = sym.Matrix([[gr**2]])
#Noise on measurements
gq =  sym.symbols('q')
gQ = sym.Matrix([[gq**2]])
gDyn_sys = Dynamic_System(gX_init, gA, gB, gR, gC, gQ, lcl_verbose=0)
#Create a single input
gu = sym.symbols('u')
gsingle_input = repeat_input(sym.Matrix([[gu]]),1)
#Initial covariance matrix
gsigma_init = sym.symbols('si')
gSigma_Init =  sym.Matrix([[gsigma_init]])
#Call the raw calman filter
gKal = Kalman_Filter(gDyn_sys, gSigma_Init, gX_init, gR, gQ, lcl_verbose=g_verbose)

#gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR, lcl_C=gC, lcl_Q=gQ, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(gsingle_input)



KALMAN FILTER CALCULATION

  Current Input Matrix([[u]])

  Predicted X Matrix([[u + xi]])
  Predicted Covariance Matrix([[q**2 + si]])
  Innovation Covariance  Matrix([[q**2 + r**2 + si]])
  Updated Kalman Gain Matrix([[(q**2 + si)/(q**2 + r**2 + si)]])
  Updated (a posteriori) estimate of covariance Matrix([[(q**2 + si)*(-(q**2 + si)/(q**2 + r**2 + si) + 1)]])
Final Values
 X                           Matrix([[u + xi + (r**2)**0.5]])
 Kalman Estimate of State X  Matrix([[u + xi + (q**2 + si)*((q**2)**0.5 + (r**2)**0.5)/(q**2 + r**2 + si)]])
 Final Covariance S(igma)    Matrix([[(q**2 + si)*(-(q**2 + si)/(q**2 + r**2 + si) + 1)]])
 Final Kalman Gains K        Matrix([[(q**2 + si)/(q**2 + r**2 + si)]])


No dynamic noise (dynamics are perfect) Kalman update becomes just the prediction in the limit

Default values for next set of experiments

In [17]:
ginput_list = repeat_input(sym.Matrix([[1],  [1]]),100)
gsingle_input = repeat_input(sym.Matrix([[1],  [1]]),1)
gX_init  = sym.Matrix.zeros(2, 1)
gSigma_Init =  sym.Matrix([[.1, 0], [0, .1]])

# Kalman Filter Impact of Noise On Gains

In the next couple of sections we will explore what happens to the Kalman Filter with time

As the Wiki points out you can express the X update in a more intuitive form

$\hat x = (I-KH)\hat x_{k-1} + K(Hx + v)$

Where we see a linear combination of the filtered state and the measured value

https://en.wikipedia.org/wiki/Kalman_filter
Equal Noise


## Equal Noise

In [22]:
#We expect this to result in equal gains and an approximately correct prediction
#it is only approximately correct because the noise on the state and on the observation mean we cannot extract the exact values
#R is the dynamic noise (error)

#These are repeated to make this stand alone
gX_init  = sym.Matrix.zeros(2, 1)
gSigma_Init =  sym.Matrix([[.1, 0], [0, .1]])

ginput_list = repeat_input(sym.Matrix([[1],  [1]]),100)

#These are generally constant but also repeated to make the cell stand alone
gA = sym.Matrix([[1, 0], [0, 1]])
gB = sym.Matrix([[1, 0], [0, 1]])
gC = sym.Matrix([[1, 0], [0, 1]])

#These are problem to problem
gQ = sym.Matrix([[0.01, 0], [0, 0.01]])
gR = sym.Matrix([[0.01, 0], [0, 0.01]])

#Q is observation noise
gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR, lcl_C=gC, lcl_Q=gQ, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(ginput_list)
#gKal.update(gsingle_input)
#With initial definition of noise
#0.00618
#With sqrt

(2, 1)
[[0.01618034 0.        ]
 [0.         0.01618034]]
[[0.02618034 0.        ]
 [0.         0.02618034]]
<class 'numpy.matrix'>
Asymptotic Covariance Matrix
[[0.00618034 0.        ]
 [0.         0.00618034]]
Asymptotic Kalman Gain K 
[[0.61803399 0.        ]
 [0.         0.61803399]]
Asymptotic P
[[0.01618034 0.        ]
 [0.         0.01618034]]
Final Actual State X
Matrix([[100.428639710877], [100.428639710877]])
Kalman Estimate of State X
Matrix([[100.404052022898], [100.404052022898]])
Final Covariance S(igma)
Matrix([[0.00618033988749895, 0], [0, 0.00618033988749895]])
Final Kalman Gains K
Matrix([[0.618033988749895, 0], [0, 0.618033988749895]])


No measurement noise (measurement is perfect) Kalman update becomes filtered prediction in the limit

In [69]:
#We expect this to result in a Kalman gain of about 1 where we are just taking the measurement
#Since there is no noise we can extract the state exactly
#Measurement noise zero
gQ1 = gQ.subs('qx', gDef_Noise*0.00).subs('qy', gDef_Noise*0)
#Dynamic noise default
gR1 = gR.subs('rx', gDef_Noise).subs('ry', gDef_Noise)

gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(ginput_list)


(2, 1)
Final Actual State 
Matrix([[99.9385165642261], [99.9385165642261]])
Kalman Estimate of State
Matrix([[99.9385165642261], [99.9385165642261]])
Final Covariance
Matrix([[0, 0], [0, 0]])
Final Kalman Gains
Matrix([[1.00000000000000, 0], [0, 1.00000000000000]])


In [70]:
#Here we make the first state perfectly observable and the second not
#The dyamic error on the first state is not zero, but on the second state it is
#Here we see the expected first state with very high Kalman gain and the second with very low
gQ1 = gQ.subs('qx', gDef_Noise*0.00).subs('qy', gDef_Noise)
#Dynamic noise default
gR1 = gR.subs('rx', gDef_Noise).subs('ry', gDef_Noise*0)

gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(ginput_list)


(2, 1)
Final Actual State 
Matrix([[99.9385165642261], [100]])
Kalman Estimate of State
Matrix([[99.9385165642261], [99.9996330418354]])
Final Covariance
Matrix([[0, 0], [0, 9.99000999000999e-5]])
Final Kalman Gains
Matrix([[1.00000000000000, 0], [0, 0.00999000999000999]])


There is of course a mathematical issue if all the noise is zero.  The Kalman gain is the inverse of $C*S*C^T+Q$ clearly if the Kalman gain is singular the first term can become singular.  However, only if the assumed measurement noise is singular will the whole expression be guaranteed to be singular.  If the measurement noise is zero you DON'T need a Kalman filter.  The obvious answer is the state is the measurement.

In [71]:
gQ1 = gQ.subs('qx', gDef_Noise*0.00).subs('qy', gDef_Noise*0.0)
#Dynamic noise default
gR1 = gR.subs('rx', gDef_Noise*0.0).subs('ry', gDef_Noise*0)

gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(gsingle_input)


(2, 1)
Final Actual State 
Matrix([[1], [1]])
Kalman Estimate of State
Matrix([[1], [1]])
Final Covariance
Matrix([[0, 0], [0, 0]])
Final Kalman Gains
Matrix([[1.00000000000000, 0], [0, 1.00000000000000]])


One other thing to look at is what the covariance converges to.  Given noise of 0.01 

In [74]:
gR1 = gR.subs('rx', gDef_Noise).subs('ry', gDef_Noise)
#Q is observation noise
gQ1 = gQ.subs('qx', gDef_Noise).subs('qy', gDef_Noise)
gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(ginput_list)

(2, 1)
Final Actual State 
Matrix([[99.9385165642261], [99.9385165642261]])
Kalman Estimate of State
Matrix([[99.9509629226324], [99.9509629226324]])
Final Covariance
Matrix([[0.00618033988749895, 0], [0, 0.00618033988749895]])
Final Kalman Gains
Matrix([[0.618033988749895, 0], [0, 0.618033988749895]])


In [75]:
gQ1 = gQ.subs('qx', gDef_Noise*10).subs('qy', gDef_Noise*10)
#Dynamic noise default
gR1 = gR.subs('rx', gDef_Noise*10).subs('ry', gDef_Noise*10)
gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(gsingle_input)

(2, 1)
Final Actual State 
Matrix([[1.09417154046807], [1.09417154046807]])
Kalman Estimate of State
Matrix([[0.970914126950111], [0.970914126950111]])
Final Covariance
Matrix([[0.0666666666666667, 0], [0, 0.0666666666666667]])
Final Kalman Gains
Matrix([[0.666666666666667, 0], [0, 0.666666666666667]])


What about steady state convergence (note this is another common notation where W=R and V=Q, note we don't need the B matrix here

$\Sigma_{t+1} =A\Sigma A^T + W - A\Sigma C^T(C\Sigma C^T+V)^{-1}C\Sigma A^T$

https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_discrete_are.html

$A\Sigma A^T - \Sigma + W - A\Sigma C^T(C\Sigma C^T+V)^{-1}C\Sigma A^T$

or from

$A^HXA - X - A^HXB(R+B^HXB)^{-1}(B^HXA)+ Q$

Clearly X=Sigma A=A B=C R=R and Q=Q

https://laurentlessard.com/teaching/me7247/lectures/lecture%2012%20-%20steady-state%20Kalman%20filter.pdf

From SciPy


https://ocw.mit.edu/courses/2-160-identification-estimation-and-learning-spring-2006/825d8851ab6950cebfab991c3e435f90_lecture_7.pdf

scipy.linalg.solve_discrete_are(a, b, q, r)[source]¶


So it seems like we should be able to predetermine the value of the Kalman filter, that should also be a check on our implementation see https://stanford.edu/class/ee363/lectures/kf.pdf

Indeed this is the case 

$\Sigma_x=A \Sigma_x A^T + B \Sigma_u B^T$

Scipi provides the solution to continious lyapunov equation

$AX + XA^H = Q$

Scipi provides the solution to and the discrete 

$AXA^H -X + Q = 0$
https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.solve_discrete_lyapunov.html#scipy.linalg.solve_discrete_lyapunov



To make these equivalent we use
$-A^{-1} \Sigma_x + \Sigma_x A^T = -A^{-1}*B*\Sigma_u*B^T$

Convieniently with our A and B both identities
$\Sigma_x + \Sigma_x = \Sigma_u$


https://spinlab.wpi.edu/courses/ece531_2013/7-5ss_kf_performance.pdf

https://spinlab.wpi.edu/courses/ece531_2013/7-5ss_kf_performance.pdf


$K = PH^T(HPH^T + R)^{-1}$

$S = P - KHP$


In [10]:
gQ11 = np.matrix(gQ.subs('qx', gDef_Noise).subs('qy', gDef_Noise)).astype(np.float64)
gR11 = np.matrix(gR.subs('rx', gDef_Noise).subs('ry', gDef_Noise)).astype(np.float64)
gA1 = np.matrix(gA).astype(np.float64)
gB1 = np.matrix(gB).astype(np.float64)
gC1 = np.matrix(gC).astype(np.float64)
print(gA1)
print(gQ11)
#gCL=linalg.solve_continuous_lyapunov(gA1, gQ11)
#gDL=linalg.solve_discrete_lyapunov(gA1, gQ11, 'bilinear')
print(gA1)
print(gC1)
print(gQ11)
print(gR11)
gP = linalg.solve_discrete_are(gA1, gC1, gQ11, gR11)
#Change of notation
gFn1 = gA1
gGn1 = gB1
gHn1 = gC1
gRn1 = gR11
gQn1 = gQ11
gK = gP*gHn1.T*((gHn1*gP*gHn1.T + gRn1).I)
print("d1")
print(gP*gHn1.T)
print(type(gP*gHn1.T))
print("d2")
print(gHn1*gP*gHn1.T + gRn1)
gS = gP-gK*gHn1*gP
gP = gFn1*gS*gFn1.T + gGn1*gQn1*gGn1.T

print("K")
print(gK)
print("S")
print(gS)
print("P")
print(gP)


[[1. 0.]
 [0. 1.]]
[[0.01 0.  ]
 [0.   0.01]]
[[1. 0.]
 [0. 1.]]
[[1. 0.]
 [0. 1.]]
[[0.01 0.  ]
 [0.   0.01]]
[[0.01 0.  ]
 [0.   0.01]]
d1
[[0.01618034 0.        ]
 [0.         0.01618034]]
<class 'numpy.matrix'>
d2
[[0.02618034 0.        ]
 [0.         0.02618034]]
K
[[0.61803399 0.        ]
 [0.         0.61803399]]
S
[[0.00618034 0.        ]
 [0.         0.00618034]]
P
[[0.01618034 0.        ]
 [0.         0.01618034]]


In [96]:
gSigma_Init =  sym.Matrix([[0.005, 0], [0, 0.005]])
gQ1 = gQ.subs('qx', gDef_Noise).subs('qy', gDef_Noise)
#Dynamic noise default
gR1 = gR.subs('rx', gDef_Noise).subs('ry', gDef_Noise)
gKal = matched_system(lcl_A=gA, lcl_B=gB, lcl_X=gX_init, lcl_R=gR1, lcl_C=gC, lcl_Q=gQ1, lcl_Sigma=gSigma_Init, lcl_verbose=1)
gKal.update(ginput_list)

(2, 1)
Final Actual State 
Matrix([[99.9385165642261], [99.9385165642261]])
Kalman Estimate of State
Matrix([[99.9509629226324], [99.9509629226324]])
Final Covariance
Matrix([[0.00618033988749895, 0], [0, 0.00618033988749895]])
Final Kalman Gains
Matrix([[0.618033988749895, 0], [0, 0.618033988749895]])


Come back to this it mentions adjoint method so interesting to see what adjoint means here
https://byjus.com/maths/symmetric-matrix/#:~:text=Some%20of%20the%20symmetric%20matrix%20properties%20are%20given,inverse%20of%20a%20transpose%20matrix.%20More%20items...%20 